<a href="https://colab.research.google.com/github/uwtintres/robotic_surgeries/blob/main/Fuzzy_Flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project is the work of Alina Saduova, CSS graduate student at UWT, Spring'21.

This notebook illustrates the implementation of the frontend Flask web application for the whole testing and simulation.

Firstle, we start with accessing the path of the colab notebooks in the Google Drive in order to be able to call all the notebooks that take part in the simulation. You may change the path to yours.


In [ ]:
path_to_col = /content/drive/MyDrive/Colab Notebooks/
%cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


Later, it is important to download all the requirements we need to launch flask app.

In [ ]:
!pip install flask
!pip install flask-ngrok
!pip install import-ipynb

Checking if we are in the right directory before launching.

In [ ]:
pwd

'/content/drive/MyDrive/Colab Notebooks'

Then, calling the main code to create a Flask web app that is taking the weights from the user on the web app, calls to all of the methods: TOPSIS, PROMETHEE II, TOPSIS with AHP, Fuzzy TOPSIS to showcase the rankings.

In [ ]:
import pandas as pd
import import_ipynb
import Fuzzy as fz
import TOPSIS as tp
import AHP as ahp
import Promethee as prm
import numpy as np
import itertools
from google.colab import files
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from random import randrange
from flask import Flask, render_template, redirect, url_for, request, make_response
from flask_ngrok import run_with_ngrok
import datetime

# defining the template folder for html template, static folder for the graphs
app = Flask(__name__, template_folder=path_to_col + '/FuzzyCode/templates', static_folder=path_to_col+'/FuzzyCode/static')
# running the web by creating tunnel and generating url via ngrok
run_with_ngrok(app)

#Renders template as soon as the application runs
@app.route('/')
def hello_world():
    return render_template('demo1.html')

# REST API to get the best alternative
@app.route('/bestsmp11', methods=['POST'])
def login():
	datafromjs = request.form['mydata']
	res = callMethods(datafromjs)
	print("res", res)

	#Creating Response to send
	resp = make_response(res)
	resp.headers['Content-Type'] = "application/json"
	return resp

# This function creates the 3D graphs and stores in the static folder
def createGraph(perf_score, vars1, vars2, sh_n):
  x = vars1
  y = vars2
  z = perf_score
  fig = plt.gcf()
  fig.set_size_inches(16, 12)
  ax = plt.axes(projection='3d')
  ax.scatter(x, y, z,
              linewidths=1, alpha=.7,
              edgecolor='k',
              s=200,
              c=z)
  # the lables of the graph change depending on the input sheet
  if sh_n == 1:
    ax.set_xlabel('Risk of cancer')
    ax.set_ylabel('Blood loss')
  elif sh_n == 2:
    ax.set_xlabel('Distance to portal vein')
    ax.set_ylabel('Imaging')
  else:
    ax.set_xlabel('Amount of mucosa')
    ax.set_ylabel('Bleeding rate')
  ax.set_zlabel('TOPSIS Performance Score')

  # Path of location to store graph
  name = '/static/graph_very_new'+datetime.datetime.now().strftime("%s")+'.png'
  fig.savefig(path_to_col+'/FuzzyCode'+name)
  return name

# This function executes all methods and returns the top 5 alternatives
def callMethods(datafromjs):
  d = datafromjs.split(",")	
  w = [float(d[0]), float(d[1]), float(d[2]), float(d[3])]
  print('weights w', w)

  # calculating TOPSIS results
  t = tp.Topsis(input_matrices[d[5]], w, new_criterias_list[d[5]])
  t.calc()
  topsis_ranks = t.rank_to_preformance_score()
  topsis_p_s = t.get_perf_score()

  # calculating AHP results and using them for TOPSIS
  t_ahp = tp.Topsis(input_matrices[d[5]], ahp_weights[d[4]], new_criterias_list[d[5]])
  t_ahp.calc()
  topsis_ahp_ranks = t_ahp.rank_to_preformance_score()
  topsis_ahp_p_s = t_ahp.get_perf_score()
  
  # calculating PROMETHEE results
  p = prm.Promethee(input_matrices[d[5]], w, new_criterias_list[d[5]])
  p.calc()
  promethee_ranks = p.rank_to_preformance_score()
  promethee_p_s = p.get_perf_score()

  # calculating Fuzzy TOPSIS results
  fuzzy_weights = getFuzzyWeights(w)
  f = fz.FuzzyTopsis(fuzzy_matrices[d[5]], fuzzy_weights, new_criterias_list[d[5]])
  f.calc()
  fuzzy_ranks = f.rank_to_closeness_coefficient()
  fuzzy_p_s = f.get_closeness_coefficient()

  # Appending 5 best results
  best_df = np.empty(shape=((len(input_matrices[d[5]]) + 4), (len(input_matrices[d[5]][0]) + 4)))
  for i in range(5):
    row = itertools.chain(t.normalized_decision[topsis_ranks[i] - 1],\
                          [topsis_p_s[topsis_ranks[i] - 1]],[topsis_ahp_p_s[topsis_ranks[i]-1]],\
                          [promethee_p_s[topsis_ranks[i]-1]],[fuzzy_p_s[topsis_ranks[i]-1]])
    best_df[i] = list(row)

  s = ""
  for row in best_df:
    for e in row:
      s += str(e)+" "
    s += ","
  
  # Calculating correlation coefficients
  correlation = []
  correlation.append(np.corrcoef(topsis_ranks, topsis_ahp_ranks)[0, 1])
  correlation.append(np.corrcoef(topsis_ranks, promethee_ranks)[0, 1])
  correlation.append(np.corrcoef(topsis_ahp_ranks, promethee_ranks)[0, 1])
  correlation.append(np.corrcoef(topsis_ahp_ranks, fuzzy_ranks)[0, 1])
  correlation.append(np.corrcoef(promethee_ranks, fuzzy_ranks)[0, 1])
  correlation.append(np.corrcoef(topsis_ranks, fuzzy_ranks)[0, 1])
  for el in correlation:
    s += str(el) + " "
  s += ","

  # providing the x and y axes information depending on the input sheet
  if (d[5] == 'Beginning'):
    attribute1 = t.normalized_decision[:, 0]
    attribute2 = t.normalized_decision[:, 1]
    sh_number = 1
  elif (d[5] == 'Middle'):
    attribute1 = t.normalized_decision[:, 1]
    attribute2 = t.normalized_decision[:, 3]
    sh_number = 2
  else:
    attribute1 = t.normalized_decision[:, 1]
    attribute2 = t.normalized_decision[:, 2]
    sh_number = 3

  name = createGraph(topsis_p_s, attribute1, attribute2, sh_number)
  s += name

  return s

# this method calls AHP to get the correlation of the attributes and to generate weights
def getAhpWeights(ahp_filename, ahp_sheetnames=[0]):
  weights_list = {}
  t = ahp.AHP()
  for ahp_sheetname in ahp_sheetnames:
    xl = pd.read_excel(ahp_filename, sheet_name=ahp_sheetname)
    xl = pd.read_excel(ahp_filename, sheet_name=ahp_sheetname, 
                    usecols=range(1, len(xl.columns)+1))
    ahp_input_matrix = xl.values.tolist()
    weights = t.calcAHP(ahp_input_matrix)
    weights_list[ahp_sheetname] = weights
  return weights_list

# this method translates quantitative weights from the web app to qualitative data for Fuzzy TOPSIS
def getFuzzyWeights(weights):
  for i in range(len(weights)):
    if 0 <= weights[i] <= 0.19:
      weights[i] = 'very low'
    elif 0.2 <= weights[i] <= 0.39:
      weights[i] = 'low'
    elif 0.4 <= weights[i] <= 0.59:
      weights[i] = 'average'
    elif 0.6 <= weights[i] <= 0.79:
      weights[i] = 'high'
    else:
      weights[i] = 'very high'
  return weights

# this method gets the input data depending on the input sheet name
def getMatrices(filename, tp_sheetnames):
  new_criterias_list = {}
  input_matrices = {}

  for tp_sheetname in tp_sheetnames:
    xl = pd.read_excel(filename, sheet_name=tp_sheetname)
    xl = pd.read_excel(filename, sheet_name=tp_sheetname, 
                        usecols=range(1, len(xl.columns)+1))
    
    # initializing input matrix, weights, criterias
    input_matrix = xl.tail(n=len(xl.index) - 2).values.tolist()
    criterias = xl.head(n=0).columns.values
    new_criterias = []
    for criteria in criterias:
      new_criterias.append(False if criteria[0] == 'N' else True)
    new_criterias_list[tp_sheetname] = new_criterias
    input_matrices[tp_sheetname] = input_matrix

  print('input matrices: ', input_matrices)
  print('criterias: ', new_criterias_list)
  return input_matrices, new_criterias_list
  
if __name__ == '__main__':
  # setting the paths for the input files and their sheetnames
  topsis_filename = path_to_col+"/FuzzyCode/TOPSIS with custom weights.xlsx"
  tp_sheetnames = ['Beginning', 'Middle', 'Ending']
  input_matrices, new_criterias_list = getMatrices(topsis_filename, tp_sheetnames)
  
  fuzzy_filename = path_to_col+"/FuzzyCode/Fuzzy TOPSIS with custom weights.xlsx"
  fuzzy_matrices, fuzzy_criterias = getMatrices(fuzzy_filename, tp_sheetnames)

  ahp_importance_filename = path_to_col+"/FuzzyCode/AHP Importance.xlsx"
  sheetnames = ['Beginning_bl', 'Beginning_rc', 'Beginning_general', 'Middle_dist', 'Middle_vision', 'Ending_muc', 'Ending_br']
  ahp_weights = getAhpWeights(ahp_importance_filename, sheetnames)
  app.run()


